In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
import torch
from torch import nn
from torch.utils.data import DataLoader
from sklearn.preprocessing import StandardScaler

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
train = pd.read_csv('../input/ingv-data/Train.csv')
times = train.iloc[:,1]
times

#final shape of training data: 4431 x 75

# ![](http://www.ready.gov/sites/default/files/2019-09/volcano.jpg)**Step 1: Feature Engineering**

In [ ]:
missing_values_count = train.isnull().sum()
missing_values_count

In [ ]:
train_frags = glob.glob("../input/predict-volcanic-eruptions-ingv-oe/train/*")
test_frags = glob.glob("../input/predict-volcanic-eruptions-ingv-oe/test/*")
check = pd.read_csv('../input/predict-volcanic-eruptions-ingv-oe/train/2037160701.csv')
check

In [ ]:
sensors = set()
observations = set()
nan_columns = list()
missed_groups = list()
for_df = list()

for item in train_frags:
    name = int(item.split('.')[-2].split('/')[-1])
    at_least_one_missed = 0
    frag = pd.read_csv(item)
    missed_group = list()
    missed_percents = list()
    for col in frag.columns:
        missed_percents.append(frag[col].isnull().sum() / len(frag))
        if pd.isnull(frag[col]).all() == True:
            at_least_one_missed = 1
            nan_columns.append(col)
            missed_group.append(col)
    if len(missed_group) > 0:
        missed_groups.append(missed_group)
    sensors.add(len(frag.columns))
    observations.add(len(frag))
    for_df.append([name, at_least_one_missed] + missed_percents)

In [ ]:
absent_df = pd.DataFrame(absent_groups.items(), columns=['Group', 'Missed number'])
absent_df = absent_df.sort_values('Missed number')

fig = px.bar(
    absent_df, 
    y="Group",
    x='Missed number',
    orientation='h',
    width=800,
    height=600,
    title='Number of missed sensor groups in training dataset'
)

fig.show()

In [ ]:
for_df = pd.DataFrame(
    for_df, 
    columns=[
        'segment_id', 'has_missed_sensors', 'missed_percent_sensor1', 
        'missed_percent_sensor2', 'missed_percent_sensor3', 'missed_percent_sensor4', 
        'missed_percent_sensor5', 'missed_percent_sensor6', 'missed_percent_sensor7', 
        'missed_percent_sensor8', 'missed_percent_sensor9', 'missed_percent_sensor10'
    ]
)

for_df

In [ ]:
train = pd.merge(train, for_df)
train

In [ ]:
def build_features(signal, ts, sensor_id):
    X = pd.DataFrame()
    f = np.fft.fft(signal)
    f_real = np.real(f)
    X.loc[ts, f'{sensor_id}_sum']       = signal.sum()
    X.loc[ts, f'{sensor_id}_mean']      = signal.mean()
    X.loc[ts, f'{sensor_id}_std']       = signal.std()
    X.loc[ts, f'{sensor_id}_var']       = signal.var() 
    X.loc[ts, f'{sensor_id}_max']       = signal.max()
    X.loc[ts, f'{sensor_id}_min']       = signal.min()
    X.loc[ts, f'{sensor_id}_skew']      = signal.skew()
    X.loc[ts, f'{sensor_id}_mad']       = signal.mad()
    X.loc[ts, f'{sensor_id}_kurtosis']  = signal.kurtosis()
    X.loc[ts, f'{sensor_id}_quantile99']= np.quantile(signal, 0.99)
    X.loc[ts, f'{sensor_id}_quantile95']= np.quantile(signal, 0.95)
    X.loc[ts, f'{sensor_id}_quantile85']= np.quantile(signal, 0.85)
    X.loc[ts, f'{sensor_id}_quantile75']= np.quantile(signal, 0.75)
    X.loc[ts, f'{sensor_id}_quantile55']= np.quantile(signal, 0.55)
    X.loc[ts, f'{sensor_id}_quantile45']= np.quantile(signal, 0.45) 
    X.loc[ts, f'{sensor_id}_quantile25']= np.quantile(signal, 0.25) 
    X.loc[ts, f'{sensor_id}_quantile15']= np.quantile(signal, 0.15) 
    X.loc[ts, f'{sensor_id}_quantile05']= np.quantile(signal, 0.05)
    X.loc[ts, f'{sensor_id}_quantile01']= np.quantile(signal, 0.01)
    X.loc[ts, f'{sensor_id}_fft_real_mean']= f_real.mean()
    X.loc[ts, f'{sensor_id}_fft_real_std'] = f_real.std()
    X.loc[ts, f'{sensor_id}_fft_real_max'] = f_real.max()
    X.loc[ts, f'{sensor_id}_fft_real_min'] = f_real.min()

    return X

In [ ]:
train_set = list()
j=0
for seg in train.segment_id:
    signals = pd.read_csv(f'/kaggle/input/predict-volcanic-eruptions-ingv-oe/train/{seg}.csv')
    train_row = []
    if j%500 == 0:
        print(j)
    for i in range(0, 10):
        sensor_id = f'sensor_{i+1}'
        train_row.append(build_features(signals[sensor_id].fillna(0), seg, sensor_id))
    train_row = pd.concat(train_row, axis=1)
    train_set.append(train_row)
    j+=1

train_set = pd.concat(train_set)

In [ ]:
train_set = train_set.reset_index()
train_set = train_set.rename(columns={'index': 'segment_id'})
train_set = pd.merge(train_set, train, on='segment_id')
train_set

In [ ]:
drop_cols = list()
for col in train_set.columns:
    if col == 'segment_id':
        continue
    if abs(train_set[col].corr(train_set['time_to_eruption'])) < 0.01:
        drop_cols.append(col)

In [ ]:
not_to_drop_cols = list()

for col1 in train_set.columns:
    for col2 in train_set.columns:
        if col1 == col2:
            continue
        if col1 == 'segment_id' or col2 == 'segment_id': 
            continue
        if col1 == 'time_to_eruption' or col2 == 'time_to_eruption':
            continue
        if abs(train_set[col1].corr(train_set[col2])) > 0.98:
            if col2 not in drop_cols and col1 not in not_to_drop_cols:
                drop_cols.append(col2)
                not_to_drop_cols.append(col1)

In [ ]:
train = train_set.drop(['segment_id', 'time_to_eruption'], axis=1)
y = train_set['time_to_eruption']

In [ ]:
reduced_y = y.copy()
reduced_train = train.copy()
reduced_train = reduced_train.drop(drop_cols, axis=1)
reduced_train

In [ ]:
train, val, y, y_val = train_test_split(train, y, random_state=666, test_size=0.2, shuffle=True)
reduced_train, reduced_val, reduced_y, reduced_y_val = train_test_split(reduced_train, reduced_y, random_state=666, test_size=0.2, shuffle=True)

In [ ]:
trainloader = torch.utils.data.DataLLoader(reduced_train, batch_size=32, shuffle=True) 

# Autoencoder for Compression

In [ ]:
class AE(torch.nn.module):
    
    def __init__(self, output_units): 
        
        super().__init__()
        self.encoder = torch.nn.Sequential(
        torch.nn.ReLu(75,36),
        torch.nn.ReLu(36,18)            
        )
        
        self.decoder = torch.nn.Sequentual(
        torch.nn.ReLu(18,36),
        torch.nn.Sigmoid(36,output_units)      
        )
        
def call(self, inputs):
    
    encoded = self.encoder(inputs)
    decoded = self.decoder(encoded)
    return decoded

auto_encoder = AE(len(reduced_train))
loss_fn = torch.nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(),
                             lr = 1e-1,
                             weight_decay = 1e-8)


In [ ]:
for epoch in range(20): 
    for i, data in enumerate(trainloader, 0): 
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        


encoder_layer = auto_encoder.get_layer('sequential')
reduced_df = pd.DataFrame(encoder_layer.predict(reduced_train))
reduced_df = reduced_df.add_prefix('feature_')

# ![](http://cdn.britannica.com/34/231234-050-5B2280BB/volcanic-eruption-Antigua-Guatemala-volcano.jpg) Step 2: The Model

In [ ]:
class Net(nn.Module):
 
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Linear(,16),
      nn.ReLU(16,8),
      nn.Linear(),
      nn.ReLU(),
      nn.Linear(1)
    )
    self.dropout = nn.dropout(0.25)


  def forward(self, X):
    X = self.layers(X)
    X = self.dropout(X)

In [ ]:
mlp = MLP()
loss_fn = nn.L1Loss()
optimizer = torch.optim.Adam(mlp.parameters(), lr=1e-4)

In [ ]:
for epoch in range(0, 5): 
    
    print(f'Starting epoch {epoch+1}')
    
    current_loss = 0.0
    
    for i, data in enumerate(trainloader, 0):
      
      inputs, targets = data
      inputs, targets = inputs.float(), targets.float()
      targets = targets.reshape((targets.shape[0], 1))
      
      optimizer.zero_grad()
      
      outputs = mlp(inputs)
      
      loss = loss_function(outputs, targets)
      
      loss.backward()
      
      optimizer.step()
      
      current_loss += loss.item()
      if i % 10 == 0:
          print('Loss after mini-batch %5d: %.3f' %
                (i + 1, current_loss / 500))
          current_loss = 0.0

  print('Training process has finished.')